<a href="https://colab.research.google.com/github/Itaigiliutin/Hishuvit3/blob/main/part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-up

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.applications import VGG19

import matplotlib.pyplot as plt
import numpy as np

In [ ]:

IMAG_SIZE = 224 # All images will be resized to 224x224
def normalize_img(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMAG_SIZE, IMAG_SIZE))
  return  image, label

In [ ]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    name="oxford_flowers102",
    split=['train', 'validation', 'test'],
    with_info=True,
    as_supervised=True,
    batch_size=32
)

train = raw_train.map(normalize_img)
#train = tfds.as_dataframe(raw_train.take(5),metadata)
validation = raw_validation.map(normalize_img)
test = raw_test.map(normalize_img)

In [ ]:
data = tf.data.TFRecordDataset("oxford_flowers102")

In [ ]:
input_shape = (224,224,3)
pixels = 224

In [ ]:
BATCH_SIZE = 16
EPOCHS = 20

steps_per_epoch = metadata.splits["train"].num_examples / BATCH_SIZE 
validation_steps = metadata.splits["validation"].num_examples / BATCH_SIZE

# Unbatch datasets to avoid batch mismatch
train = train.unbatch().batch(BATCH_SIZE,drop_remainder=True).repeat().shuffle(512)
validation = validation.unbatch().batch(BATCH_SIZE,drop_remainder=True).repeat()

# Model Training

## MobileNet

In [ ]:
mobileNet_accuracy = 0
for alpha in np.linspace(10,40,10):
  mobileNet_model_alpha = tf.keras.Sequential([
      hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5", trainable=True),
      tf.keras.layers.Dropout(rate=(alpha/100)),
      tf.keras.layers.Dense(units=metadata.features["label"].num_classes)
  ])

  mobileNet_model_alpha.build((None,pixels, pixels,3))
  mobileNet_model_alpha.summary()

  mobileNet_model_alpha.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=.5e-3), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

  history_alpha = mobileNet_model_alpha.fit(
      x=train,
      epochs=EPOCHS, 
      steps_per_epoch=steps_per_epoch,
      validation_data=validation,
      validation_steps=validation_steps,
      verbose=2)
  
  score_alpha = mobileNet_model_alpha.evaluate(x=test)
  accuracy_alpha = score_alpha[1]
  if(accuracy_alpha > mobileNet_accuracy):
    (mobileNet_model, mobileNet_history, mobileNet_score, mobileNet_accuracy) = (mobileNet_model_alpha, history_alpha, score_alpha, accuracy_alpha) 

In [ ]:
print("loss:", mobileNet_score[0])
print("accuracy:", mobileNet_score[1])

## nasNet

In [ ]:
model = NASNetMobile(include_top = False, pooling = 'avg', weights = 'imagenet',input_shape=input_shape)
size = len(model.layers)
for idx,layer in enumerate(model.layers):
  if idx < size/2:
    layer.trainable = False
  else:
    layer.trainable = True
nasNet_model = keras.Sequential([
    model,
    tf.keras.layers.Dropout(rate=0.25),
    tf.keras.layers.Dense(units=metadata.features["label"].num_classes)
])
nasNet_model.build((None,pixels, pixels,3))
print(nasNet_model.summary())

In [ ]:
nasNet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=.5e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=["accuracy"])

history = nasNet_model.fit(
    x=train,
    epochs=EPOCHS, 
    steps_per_epoch=steps_per_epoch,
    validation_data=validation,
    validation_steps=validation_steps,
    verbose=2)

nasNet_scores = nasNet_model.evaluate(x=test, verbose=0)
print("loss:", nasNet_scores[0])
print("accuracy:", nasNet_scores[1])